## Denoising fMRI data for connectivity analysis

This script denoising fMRI task data for further PPI analysis. Denoising strategy consists of:
- confounds regression: 24 head motion parameters, white matter and csf signal (raw, sqared, derivative and square of derivative)
- detrending
- high pass filtering with cut-off 128s
- standardizing resulting signal

Denoised images are saved under `derivatives/fmridenoise` directory with BIDS compliant names.

---
**Last update**: 25.08.2020

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
import os

from nilearn.image import clean_img
from bids import BIDSLayout
import nibabel as nb

path_root = os.environ.get('DECIDENET_PATH')
path_code = os.path.join(path_root, 'code')
if path_code not in sys.path:
    sys.path.append(path_code)
from dn_utils.behavioral_models import load_behavioral_data                    

In [ ]:
t_r = 2
n_conditions = 2

path_out = os.path.join(path_root, 'data/main_fmri_study/derivatives/fmridenoise')
path_bids = os.path.join(path_root, 'data/main_fmri_study')

layout = BIDSLayout(
    root=path_bids,
    derivatives=True,
    validate=True,
    index_metadata=False
)

fmri_filter = {
    "extension": [".nii", ".nii.gz"],
    "space": "MNI152NLin2009cAsym",
    "suffix": "bold",
    "desc": "preproc",
    "return_type": "filename"
}

conf_filter = {
    "extension": "tsv",
    "desc": "confounds",
    "return_type": "filename"
}

fmri_files, conf_files = [], []

for task_dict in [{'task': 'prlrew'}, {'task': 'prlpun'}]:
    fmri_filter.update(task_dict)
    conf_filter.update(task_dict)
    fmri_files.append(layout.get(**fmri_filter))
    conf_files.append(layout.get(**conf_filter))

In [ ]:
for con in range(n_conditions):
    for fmri_file, conf_file in zip(fmri_files[con], conf_files[con]):
        
        # Read and filter confounds
        conf_df = pd.read_csv(conf_file, sep='\t')
        conf_df = conf_df.filter(regex='rot|trans|csf|white_matter')
        conf_df = conf_df.reindex(sorted(conf_df.columns), axis=1)
        conf_df = conf_df.fillna(0)
        
        # Denoise image
        fmri_denoised = clean_img(
            nb.load(fmri_file), 
            confounds=conf_df.values, 
            detrend=True,
            standardize=True,
            high_pass=1/128, 
            t_r=t_r
        )
        
        # Save denoised image
        base_substr = fmri_file.split('/')[-1].split('.')[0]
        base_substr = base_substr.replace('_desc-preproc_bold', '')
        pipeline_substr = f'pipeline-24HMPCSFWM'
        suffix = 'desc-denoised_bold.nii.gz'
        fmri_denoised_fname = f'{base_substr}_{pipeline_substr}_{suffix}'
        nb.save(fmri_denoised, os.path.join(path_out, fmri_denoised_fname))
        
        print(f'Saving {fmri_denoised_fname}')